In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
# Load the pre-trained Inception-ResNet-V2 model
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(250,250,3))

# Freeze the pre-trained weights
base_model.trainable = False

# Define the input shape
input_shape = (250, 250, 3)

# Define the input layer
inputs = tf.keras.layers.Input(shape=input_shape)

# Pass the input layer through the pre-trained model
features = base_model(inputs)

# Add a dense layer for multi-class classification
# outputs = tf.keras.layers.Dense(units=8, activation='sigmoid')(features)

outputs = tf.keras.layers.Flatten()(features)
outputs = tf.keras.layers.Dense(units=8, activation='sigmoid')(outputs)

In [3]:
import tensorflow_addons as tfa
accuracy = tf.keras.metrics.Accuracy()
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
f1_score = tfa.metrics.F1Score(num_classes=8)

In [4]:
# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)
# Compile the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
               loss='binary_crossentropy',
              metrics=['accuracy', precision , recall, f1_score])

In [5]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 6, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 55296)             0         
                                                                 
 dense (Dense)               (None, 8)                 442376    
                                                                 
Total params: 54,779,112
Trainable params: 442,376
Non-trainable params: 54,336,736
_________________________________________________________________


In [6]:
# Load the image data and one-hot encoded multi-label data
# Load the data from the excel sheet into a pandas dataframe
# Load both left and right eye dataset
base_path = 'C:/Users/User/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')

#concating both dataframes
df = pd.concat([left_df, right_df], axis=0)


# Split the data into two arrays, one for the image paths and one for the labels
images_base_path = 'E:/odir-dataset/ODIR-5K/ODIR-5K/cropped_training_images/'
image_paths = images_base_path + df["Fundus"].values
labels = df.iloc[:, 3:].values

# Convert the lists to tensors
image_paths = tf.constant(image_paths)
labels = tf.constant(labels)

# Create a dataset from the image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/DeepLearning/DL-Project/updated_left_eye.xlsx'

In [7]:
# Load the image data and one-hot encoded multi-label data
# Load the data from the excel sheet into a pandas dataframe
# Load both left and right eye dataset
base_path = 'C:/Users/manik/DL-Project/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')

#concating both dataframes
df = pd.concat([left_df, right_df], axis=0)


# Split the data into two arrays, one for the image paths and one for the labels
images_base_path = 'D:/cropped_training_images/'
image_paths = images_base_path + df["Fundus"].values
labels = df.iloc[:, 3:].values

# Convert the lists to tensors
image_paths = tf.constant(image_paths)
labels = tf.constant(labels)

# Create a dataset from the image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

In [8]:
# Load the images and preprocess them
def load_and_preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [250, 250])
    image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    return image, label

# Apply the load_and_preprocess_image function to the dataset
dataset = dataset.map(load_and_preprocess_image)

# Shuffle the data and batch it
batch_size = 32
dataset = dataset.shuffle(buffer_size=len(image_paths)).batch(batch_size)

In [10]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 8), dtype=tf.int64, name=None))>

In [9]:
def split_data(dataset, train_split, test_split, val_split):
    # Calculate the size of each split
    dataset_size = dataset.reduce(tf.constant(0, dtype=tf.int64), lambda acc, _: acc + 1).numpy()
    train_size = int(dataset_size * train_split)
    val_size = int(dataset_size * val_split)
    test_size = int(dataset_size * test_split)

    # Shuffle the elements of the dataset randomly
    dataset = dataset.shuffle(buffer_size=dataset_size, seed=42)

    # Split the dataset into train, validation, and test sets
    train_dataset = dataset.take(train_size)
    remaining_dataset = dataset.skip(train_size)
    val_dataset = remaining_dataset.take(val_size)
    test_dataset = remaining_dataset.skip(val_size)
    return train_dataset, test_dataset, val_dataset

In [10]:
# Split the data into training and validation sets
train_dataset, test_dataset, val_dataset = split_data(dataset, train_split=0.7, test_split=0.15, val_split=0.15)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [13]:
# Train the model
history=model.fit(train_dataset, epochs=10, batch_size=batch_size, validation_data=test_dataset)

Epoch 1/10
152/152 [==============================] - 1026s 6s/step - loss: 7.8826 - accuracy: 0.4421 - precision: 0.1379 - recall: 0.9877 - f1_score: 0.2124 - val_loss: 4.9353 - val_accuracy: 0.4407 - val_precision: 0.1341 - val_recall: 1.0000 - val_f1_score: 0.2136
Epoch 2/10
152/152 [==============================] - 1003s 6s/step - loss: 4.5284 - accuracy: 0.4485 - precision: 0.1340 - recall: 1.0000 - f1_score: 0.2117 - val_loss: 3.2648 - val_accuracy: 0.4301 - val_precision: 0.1344 - val_recall: 1.0000 - val_f1_score: 0.2145
Epoch 3/10
152/152 [==============================] - 907s 6s/step - loss: 4.1962 - accuracy: 0.4484 - precision: 0.1341 - recall: 1.0000 - f1_score: 0.2124 - val_loss: 5.8221 - val_accuracy: 0.4651 - val_precision: 0.1324 - val_recall: 1.0000 - val_f1_score: 0.2091
Epoch 4/10
152/152 [==============================] - 1038s 6s/step - loss: 4.7766 - accuracy: 0.4393 - precision: 0.1339 - recall: 1.0000 - f1_score: 0.2127 - val_loss: 5.5564 - val_accuracy: 0.44

In [14]:
model.save('C:/Users/User/DeepLearning/DL-Project/models/model_2.h5')

In [11]:
# Train the model
history=model.fit(train_dataset, epochs=10, batch_size=batch_size, validation_data=test_dataset)

Epoch 1/10
152/152 [==============================] - 831s 5s/step - loss: 0.3348 - accuracy: 0.4598 - precision: 0.4875 - recall: 0.3199 - f1_score: 0.3167 - val_loss: 0.2715 - val_accuracy: 0.5389 - val_precision: 0.5649 - val_recall: 0.4335 - val_f1_score: 0.4037
Epoch 2/10
152/152 [==============================] - 668s 4s/step - loss: 0.2697 - accuracy: 0.5340 - precision: 0.5953 - recall: 0.4032 - f1_score: 0.4202 - val_loss: 0.2306 - val_accuracy: 0.6131 - val_precision: 0.6887 - val_recall: 0.4418 - val_f1_score: 0.5232
Epoch 3/10
152/152 [==============================] - 815s 5s/step - loss: 0.2491 - accuracy: 0.5765 - precision: 0.6394 - recall: 0.4291 - f1_score: 0.4903 - val_loss: 0.2276 - val_accuracy: 0.5744 - val_precision: 0.7027 - val_recall: 0.4487 - val_f1_score: 0.4446
Epoch 4/10
152/152 [==============================] - 840s 5s/step - loss: 0.2322 - accuracy: 0.6063 - precision: 0.6667 - recall: 0.4805 - f1_score: 0.5387 - val_loss: 0.2089 - val_accuracy: 0.6360 

In [13]:
model.save('C:/Users/manik/DL-Project/DeepLearning/DL-Project/models/model_2_v1.h5')

In [15]:
model = keras.models.load_model('C:/Users/manik/DL-Project/DeepLearning/DL-Project/models/model_2_v1.h5')

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
               loss='binary_crossentropy',
              metrics=['accuracy', precision , recall, f1_score])

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 inception_resnet_v2 (Functi  (None, 6, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 55296)             0         
                                                                 
 dense (Dense)               (None, 8)                 442376    
                                                                 
Total params: 54,779,112
Trainable params: 442,376
Non-trainable params: 54,336,736
_________________________________________________________________


In [18]:
# Train the model
history=model.fit(train_dataset, epochs=10, batch_size=batch_size, validation_data=test_dataset)

Epoch 1/10
152/152 [==============================] - 686s 4s/step - loss: 0.1343 - accuracy: 0.8082 - precision: 0.8376 - recall: 0.6979 - f1_score: 0.7869 - val_loss: 0.1332 - val_accuracy: 0.8051 - val_precision: 0.8559 - val_recall: 0.7422 - val_f1_score: 0.8145
Epoch 2/10
152/152 [==============================] - 731s 4s/step - loss: 0.1353 - accuracy: 0.8116 - precision: 0.8702 - recall: 0.7058 - f1_score: 0.8193 - val_loss: 0.1328 - val_accuracy: 0.8226 - val_precision: 0.8813 - val_recall: 0.7504 - val_f1_score: 0.8258
Epoch 3/10
152/152 [==============================] - 1141s 7s/step - loss: 0.1318 - accuracy: 0.8172 - precision: 0.8832 - recall: 0.7241 - f1_score: 0.8291 - val_loss: 0.1290 - val_accuracy: 0.8061 - val_precision: 0.8707 - val_recall: 0.7453 - val_f1_score: 0.8495
Epoch 4/10
152/152 [==============================] - 1181s 7s/step - loss: 0.1331 - accuracy: 0.8138 - precision: 0.8850 - recall: 0.7215 - f1_score: 0.8304 - val_loss: 0.1302 - val_accuracy: 0.828

In [21]:
model.save('C:/Users/manik/DL-Project/DeepLearning/DL-Project/models/model_2_v2.h5')